# Recovery Data Partnership - SafeGraph user guide

### 1. Install Dependencies

In [2]:
! pip install s3fs pandas

     |████████████████████████████████| 92kB 4.4MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 6.5MB 7.7MB/s 
     |████████████████████████████████| 1.2MB 45.0MB/s 
     |████████████████████████████████| 552kB 49.4MB/s 
     |████████████████████████████████| 153kB 50.4MB/s 
     |████████████████████████████████| 266kB 46.1MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=7a5a9ac18ce743e5b04a754003a9e130385e18bfd9aebfd2d0a3891bc2e3a089
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16


In [4]:
import pandas as pd
import s3fs

In [5]:
s3 = s3fs.S3FileSystem(
      key='<YOUR KEY HERE>',
      secret='<YOUR SECRET HERE>', 
      client_kwargs={
          'endpoint_url': 'https://s3.amazonaws.com', 
          'region_name':'us-east-1'
      }
    )

### 2. Listing available files


In [7]:
s3.ls('recovery-data-partnership/output/')

['recovery-data-partnership/output/example',
 'recovery-data-partnership/output/lookups',
 'recovery-data-partnership/output/poi',
 'recovery-data-partnership/output/social_distancing']

In [8]:
s3.ls('recovery-data-partnership/output/social_distancing/')

['recovery-data-partnership/output/social_distancing/weekly_county_trips',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips']

In [10]:
s3.ls('recovery-data-partnership/output/social_distancing/weekly_state_trips/')

['recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2019Q1.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2019Q2.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2019Q3.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2019Q4.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2020Q1.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2020Q2.csv.zip',
 'recovery-data-partnership/output/social_distancing/weekly_state_trips/weekly_state_trips_2020Q3.csv.zip']

### 3. Reading files to dataframe and combine them together

In [18]:
dfs = []
for dataset in s3.ls('recovery-data-partnership/output/social_distancing/weekly_state_trips/'):
  df = pd.read_csv(s3.open(dataset, mode='rb'), compression='zip')
  dfs.append(df)
  del df

In [19]:
weekly_state_trips = pd.concat(dfs)

In [22]:
weekly_state_trips = weekly_state_trips.groupby(['year_week',	'state']).sum()

In [26]:
weekly_state_trips

to_nyc  from_nyc  net_nyc
year_week state                           
2019-01   1       48691     21817    26874
          2        2288      2044      244
          4       42800     56635   -13835
          5       20910     14140     6770
          6      364721    507026  -142305
...                 ...       ...      ...
2020-40   54       3038      8467    -5429
          55       2086      7714    -5628
          56        434      3882    -3448
          72       2828     15001   -12173
          78        224      1259    -1035

[4912 rows x 3 columns]

### 4. We also prepared look up tables 

In [24]:
s3.ls('recovery-data-partnership/output/lookups/')

['recovery-data-partnership/output/lookups/',
 'recovery-data-partnership/output/lookups/fips_to_county.csv',
 'recovery-data-partnership/output/lookups/fips_to_state.csv',
 'recovery-data-partnership/output/lookups/naics_sector.csv',
 'recovery-data-partnership/output/lookups/naics_subsector.csv',
 'recovery-data-partnership/output/lookups/nta_to_boro_county.csv']